In [1]:
#!/bin/env python

# =============================================================================
# GLOBAL IMPORTS
# =============================================================================
import os
import glob
import io
import collections
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy.stats


# =============================================================================
# CONSTANTS
# =============================================================================

# Paths to input data.
LOGP_SUBMISSIONS_DIR_PATH = 'analysis/logP_predictions'
EXPERIMENTAL_DATA_FILE_PATH = './experimental_data/logP_experimental_values.csv'



In [2]:
class IgnoredSubmissionError(Exception):
    """Exception used to signal a submission that must be ignored."""
    pass


class BadFormatError(Exception):
    """Exception used to signal a submission with unexpected formatting."""
    pass


class SamplSubmission:
    """A generic SAMPL submission.
    Parameters
    ----------
    file_path : str
        The path to the submission file.
    Raises
    ------
    IgnoredSubmission
        If the submission ID is among the ignored submissions.
    """
    # The D3R challenge IDs that are handled by this class.
    CHALLENGE_IDS = {1559}

    # The IDs of the submissions used for testing the validation.
    TEST_SUBMISSIONS = {}

    # Section of the submission file.
    SECTIONS = {}

    # Sections in CSV format with columns names.
    CSV_SECTIONS = {}
    
    def __init__(self, file_path, user_map):
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        print(file_name)
        file_data = file_name.split('-')

        # Check if this is a deleted submission.
        if file_data[0] == 'DELETED':
            raise IgnoredSubmissionError('This submission was deleted.')

        # Check if this is a test submission.
        self.receipt_id = file_data[0]
        if self.receipt_id in self.TEST_SUBMISSIONS:
            raise IgnoredSubmissionError('This submission has been used for tests.')

        # Check this is the correct challenge.
        self.challenge_id = int(file_data[1])
        assert self.challenge_id in self.CHALLENGE_IDS

        # Store user map information.
        user_map_record = user_map[user_map.receipt_id == self.receipt_id]
        assert len(user_map_record) == 1
        user_map_record = user_map_record.iloc[0]

        self.id = user_map_record.id
        self.participant = user_map_record.firstname + ' ' + user_map_record.lastname
        self.participant_id = user_map_record.uid
        #self.participant_email = user_map_record.email
        #assert self.challenge_id == user_map_record.component
    
    @classmethod
    def _read_lines(cls, file_path):
        """Generator to read the file and discard blank lines and comments."""
        with open(file_path, 'r', encoding='utf-8-sig') as f:
            for line in f:
                # Strip whitespaces.
                line = line.strip()
                # Don't return blank lines and comments.
                if line != '' and line[0] != '#':
                    yield line

    @classmethod
    def _load_sections(cls, file_path):
        """Load the data in the file and separate it by sections."""
        sections = {}
        current_section = None
        for line in cls._read_lines(file_path):
            # Check if this is a new section.
            if line[:-1] in cls.SECTIONS:
                current_section = line[:-1]
            else:
                if current_section is None:
                    import pdb
                    pdb.set_trace()
                try:
                    sections[current_section].append(line)
                except KeyError:
                    sections[current_section] = [line]

        # Check that all the sections have been loaded.
        found_sections = set(sections.keys())
        if found_sections != cls.SECTIONS:
            raise BadFormatError('Missing sections: {}.'.format(found_sections - cls.SECTIONS))
            
    # Create a Pandas dataframe from the CSV format.
        for section_name in cls.CSV_SECTIONS:
            csv_str = io.StringIO('\n'.join(sections[section_name]))
            columns = cls.CSV_SECTIONS[section_name]
            id_column = columns[0]
            section = pd.read_csv(csv_str, index_col=id_column, names=columns, skipinitialspace=True)
            #section = pd.read_csv(csv_str, names=columns, skipinitialspace=True)
            sections[section_name] = section
        return sections

    @classmethod
    def _create_comparison_dataframe(cls, column_name, submission_data, experimental_data):
        """Create a single dataframe with submission and experimental data."""
        # Filter only the systems IDs in this submissions.


        experimental_data = experimental_data[experimental_data.index.isin(submission_data.index)] # match by column index
        # Fix the names of the columns for labelling.
        submission_series = submission_data[column_name]
        submission_series.name += ' (calc)'
        experimental_series = experimental_data[column_name]
        experimental_series.name += ' (expt)'

        # Concatenate the two columns into a single dataframe.
        return pd.concat([experimental_series, submission_series], axis=1)

class logPSubmission(SamplSubmission):
    """A submission for logP challenge.

    Parameters
    ----------
    file_path : str
        The path to the submission file

    Raises
    ------
    IgnoredSubmission
        If the submission ID is among the ignored submissions.

    """

    # The D3R challenge IDs that are handled by this class.
    CHALLANGE_IDS = {1559}

    # The IDs of the submissions that will be ignored in the analysis.
    TEST_SUBMISSIONS = {}

    # Section of the submission file.
    SECTIONS = {'Predictions', 'Name', 'Software', 'Category', 'Method'}

    # Sections in CSV format with columns names.
    CSV_SECTIONS = {'Predictions': ("Molecule ID", "logP mean", "logP SEM", "logP model uncertainty")}


    def __init__(self, file_path, user_map):
        super().__init__(file_path, user_map)

        file_name = os.path.splitext(os.path.basename(file_path))[0]
        file_data = file_name.split('-')

        # Check if this is a type III submission
        
        self.submission_type = file_data[2]
        assert self.submission_type in ['logP']

        self.file_name, self.index = file_data[3:]
        self.index = int(self.index)

        # Load predictions.
        sections = self._load_sections(file_path)  # From parent-class.
        self.data = sections['Predictions']  # This is a pandas DataFrame.
        self.name = sections['Name'][0]
        self.category = sections['Category'][0] # New section for logP challenge.

    def compute_logP_statistics(self, experimental_data, stats_funcs):
        data = self._create_comparison_dataframe('logP mean', self.data, experimental_data)

        # Create lists of stats functions to pass to compute_bootstrap_statistics.
        stats_funcs_names, stats_funcs = zip(*stats_funcs.items())
        bootstrap_statistics = compute_bootstrap_statistics(data.as_matrix(), stats_funcs, n_bootstrap_samples=10000)

        # Return statistics as dict preserving the order.
        return collections.OrderedDict((stats_funcs_names[i], bootstrap_statistics[i])
                                      for i in range(len(stats_funcs)))

    def compute_logP_model_uncertainty_statistics(self,experimental_data):

        # Create a dataframe for data necessary for error slope analysis
        expt_logP_series = experimental_data["logP mean"]
        expt_logP_SEM_series = experimental_data["logP SEM"]
        pred_logP_series = self.data["logP mean"]
        pred_logP_SEM_series = self.data["logP SEM"]
        pred_logP_mod_unc_series = self.data["logP model uncertainty"]

        # Concatenate the columns into a single dataframe.
        data_exp =  pd.concat([expt_logP_series, expt_logP_SEM_series], axis=1)
        data_exp = data_exp.rename(index=str, columns={"logP mean": "logP mean (expt)",
                                                        "logP SEM": "logP SEM (expt)"})
        
        data_mod_unc = pd.concat([data_exp, pred_logP_series, pred_logP_SEM_series, pred_logP_mod_unc_series], axis=1)
        data_mod_unc = data_mod_unc.rename(index=str, columns={"logP mean (calc)": "logP mean (calc)",
                                                                "logP SEM": "logP SEM (calc)",
                                                                "logP model uncertainty": "logP model uncertainty"})
        #print("data_mod_unc:\n", data_mod_unc)

        # Compute QQ-Plot Error Slope (ES)
        calc = data_mod_unc.loc[:, "logP mean (calc)"].values
        expt = data_mod_unc.loc[:, "logP mean (expt)"].values
        dcalc = data_mod_unc.loc[:, "logP model uncertainty"].values
        dexpt = data_mod_unc.loc[:, "logP SEM (expt)"].values
        n_bootstrap_samples = 1000

        X, Y, error_slope, error_slope_std, slopes = getQQdata(calc, expt, dcalc, dexpt, boot_its=n_bootstrap_samples)
        #print(X)
        #print(Y)
        #print("ES:", error_slope)
        #print("ES std:", error_slope_std)
        #print("Bootstrapped Error Slopes:", slopes)
        QQplot_data = [X, Y, error_slope]

        # Compute 95% confidence intervals of Error Slope
        percentile = 0.95
        percentile_index = int(np.floor(n_bootstrap_samples * (1 - percentile) / 2)) - 1

        #for stats_func_idx, samples_statistics in enumerate(bootstrap_samples_statistics):
        samples_statistics = np.asarray(slopes)
        samples_statistics.sort()
        stat_lower_percentile = samples_statistics[percentile_index]
        stat_higher_percentile = samples_statistics[-percentile_index + 1]
        confidence_interval = (stat_lower_percentile, stat_higher_percentile)

        model_uncertainty_statistics = [error_slope, confidence_interval, samples_statistics]
        
        return model_uncertainty_statistics, QQplot_data

In [3]:
def load_submissions(directory_path, user_map):
    submissions = []
    for file_path in glob.glob(os.path.join(directory_path, '*.csv')):
        try:
            submission = logPSubmission(file_path, user_map)

        except IgnoredSubmissionError:
            continue
        submissions.append(submission)
    return submissions

In [4]:
# =============================================================================
# SOME ANALYSIS
# =============================================================================

sns.set_style('whitegrid')
sns.set_context('paper')

# Read experimental data.
with open(EXPERIMENTAL_DATA_FILE_PATH, 'r') as f:
    # experimental_data = pd.read_json(f, orient='index')
    names = ('Molecule ID', 'logP mean', 'logP SEM',
             'Assay Type', 'Experimental ID', 'Isomeric SMILES')
    experimental_data = pd.read_csv(f, names=names, skiprows=1)

# Convert numeric values to dtype float.
for col in experimental_data.columns[1:7]:
        experimental_data[col] = pd.to_numeric(experimental_data[col], errors='coerce')


experimental_data.set_index("Molecule ID", inplace=True)
experimental_data["Molecule ID"] = experimental_data.index
print("Experimental data: \n", experimental_data)

# Import user map.
with open('../predictions/SAMPL6-user-map-logP.csv', 'r') as f:
    user_map = pd.read_csv(f)

# Load submissions data.
submissions_logP = load_submissions(LOGP_SUBMISSIONS_DIR_PATH, user_map)

Experimental data: 
              logP mean  logP SEM  Assay Type  Experimental ID  \
Molecule ID                                                     
SM02              4.09      0.03         NaN              NaN   
SM04              3.98      0.03         NaN              NaN   
SM07              3.21      0.04         NaN              NaN   
SM08              3.10      0.03         NaN              NaN   
SM09              3.03      0.07         NaN              NaN   
SM11              2.10      0.04         NaN              NaN   
SM12              3.83      0.03         NaN              NaN   
SM13              2.92      0.04         NaN              NaN   
SM14              1.95      0.03         NaN              NaN   
SM15              3.07      0.03         NaN              NaN   
SM16              2.62      0.01         NaN              NaN   

             Isomeric SMILES Molecule ID  
Molecule ID                               
SM02                     NaN        SM02  
SM04

FileNotFoundError: [Errno 2] No such file or directory: '../predictions/SAMPL6-user-map-logP.csv'

In [15]:
submissions_logP[0].data


,logP mean,logP SEM,logP model uncertainty
Molecule ID,,,
SM02,4.03,0.31,0.31
SM04,3.73,0.32,0.31
SM07,3.09,0.29,0.31
SM08,2.53,0.46,0.31
SM09,3.27,0.29,0.31
SM11,1.21,0.36,0.31
SM12,3.82,0.33,0.31
SM13,3.57,0.23,0.31
SM14,2.12,0.23,0.31


In [19]:
submissions_logP[0].data.loc['SM08']["logP mean"]

2.53

In [41]:
for item in submissions_logP:
    if float(item.data.loc['SM08']["logP mean"]) > 7:
        #print(item.name)
        print(item.name, item.data.loc['SM08']["logP mean"])

MD-OPLSAA-dryoct 8.55
YANK-GAFF-tip3p-ForceBalance-wet 9.41
YANK-GAFF-tip3p-wet 7.96
MD-OPLSAA-wetoct 7.59
